In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import analyse_file
from EC_data_processing_lib import get_decimal_separator

fig = go.Figure()
cal = 0.309 #calibration of the Hg/HgO electrode vs RHE converting to overpotential vs OER

#the following section uploads the data. You need to update the dataset array when you add or remove files. 
CoNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_05 CoNx Ti electrode1_02_CV_C05.mpt', #file path the slash needs to be '/' rather than the '\' which is automatic
'label' : 'CoN<sub>X</sub>',
}

NiNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_04 NiNx Ti electrode1_02_CV_C05.mpt',
'label' : 'NiN<sub>X</sub>',
}

FeNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_11 FeNx Ti electrode set1_02_CV_C05.mpt',
'label' : 'FeN<sub>X</sub>',
}

dataset = [CoNx_dict, NiNx_dict, FeNx_dict]

for i in dataset:
    headerlength = get_header_length(i['data'])
    data = pd.read_csv(i['data'], encoding='ansi', sep="\t", decimal=get_decimal_separator(i['data']),  skiprows=range(headerlength))
    cn = data['cycle number']
    data = data[data['cycle number']== float(cn[-1:])] # allowing this comment will plot only the last scan. If you want to plot all of the scans then comment this out. 
    voltage = data['Ewe/V']
    current = data['<I>/mA']
    voltage_corrected = data['Ewe/V'] - data['<I>/mA'] * i['Ru']/1000 - cal #does the iR correction converts to overpotential

    fig.add_trace(go.Scatter(
        x=voltage_corrected,
        y=current,
        name= i['label']
    ))
fig.update_xaxes(range=[0.00, 0.700],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[-5, 50],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    #title="CV",
    xaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)

fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Cyclic Voltammetry.pdf')

